In [1]:
import sys, os
from pathlib import Path
import time

parent_directory = Path.cwd().parent
# Add the parent directory to sys.path
# print(parent_directory)
sys.path.append(str(parent_directory))
# print(sys.path)

In [2]:
from modules.rds_access import execute_sql_query, DatabaseConnection
import os
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:

conn_params = {                                                   
        "host": os.environ.get("POSTGRE_HOST", ''),                                
        "port": 5432,                                                              
        "user": os.environ.get("POSTGRE_USER", ''),                                
        "password": os.environ.get("POSTGRE_PWD", '')                              
    }        

In [9]:
conn_params['database'] = "kugou"

sql_query = """SELECT ks.*, ka.album_name
FROM kugou_songs ks
LEFT JOIN kugou_albums ka ON ks.album_id = ka.album_id
WHERE work_name ilike '%SuperFX%'
"""
with DatabaseConnection(conn_params) as conn:
    df = execute_sql_query(sql_query, conn)
    print(df.shape)
df.head().T

(12, 15)


,0,1,2,3,4
audio_id,48408857,78767562,78767567,291406516,78767421
singer_id,None,None,None,None,None
language,None,None,None,英语,None
ori_author_name,Siriusmo,Thomas Bergersen,Thomas Bergersen,WesHxc,Thomas Bergersen
work_name,SUPERFX,SuperFX(Hit)Orchestra Hit II,SuperFX(Hit)Orchestra Hit VI,SuperFX,SuperFX(Hit)Orchestra Hit XI
cid,65216314,109253391,109253373,327163370,109253346
scrape_date,2024-03-16 19:00:31.410000+00:00,2024-03-16 21:43:45.456000+00:00,2024-03-16 21:43:45.459000+00:00,2024-03-17 08:48:12.167000+00:00,2024-03-16 21:43:45.420000+00:00
combine_count,0,1,1,0,1
album_id,17029556,24268132,24268132,79928054,24268132
publish_company,Monkeytown Records,Nemesis Productions LLC,Nemesis Productions LLC,BESCAREDrecords,Nemesis Productions LLC


In [11]:
# function to filter the corresponding lines
def row_filter(row):
    return row['ori_author_name'].lower().strip() == 'nana' and 'lonely' in row['work_name'].lower()
    

In [16]:
df_filtered = df.loc[df.apply(row_filter, axis=1)]

In [17]:
df_filtered.to_csv('result_filtered.csv')

In [28]:
start = time.time()
# query for the netease_max database. 

# sql_query = """SELECT album_id, song_id, song_name, artist_id, copyright_id, deprecated_artist_name, artist_ids,
# comment_count, audit_songs.artist_array, album_name, release_date, company, song_array
# FROM audit_songs
# INNER JOIN audit_albums USING (album_id)
# where '102714' = any(audit_songs.artist_array)
# or '792429' = any(audit_songs.artist_array)
# or '45336' = any(audit_songs.artist_array)

# """
sql_query = """
SELECT audit_songs.album_id, audit_songs.song_id, audit_songs.song_name, audit_songs.artist_id, 
copyright_id, deprecated_artist_name, artist_ids,
audit_songs.comment_count, audit_songs.artist_array, album_name, release_date, 
company, song_array, likes as likes_count, 
award.play_num as award_play_num, award.comment_count as award_comment_count, 
top_songs.play_count as top_songs_play_count
FROM audit_songs
INNER JOIN audit_albums on audit_songs.album_id = audit_albums.album_id
INNER JOIN honor_single_award as award on award.song_id = audit_songs.song_id
INNER JOIN honor_top_songs as top_songs on audit_songs.song_id::text = top_songs.song_id
where audit_songs.song_id = '31010566'
"""
# sql_query = "select * from audit_songs limit 5"
# sql_query = '''
# SELECT * 
# FROM honor_single_award
# INNER JOIN honor_top_songs ON honor_single_award.song_id::text = honor_top_songs.song_id
# WHERE honor_single_award.song_id = 31010566;

# '''
conn_params['database'] = "netease_max"
with DatabaseConnection(conn_params) as conn:
    df = execute_sql_query(sql_query, conn)
    print(sql_query)
df.head(100).T


SELECT audit_songs.album_id, audit_songs.song_id, audit_songs.song_name, audit_songs.artist_id, 
copyright_id, deprecated_artist_name, artist_ids,
audit_songs.comment_count, audit_songs.artist_array, album_name, release_date, 
company, song_array, likes as likes_count, 
award.play_num as award_play_num, award.comment_count as award_comment_count, 
top_songs.play_count as top_songs_play_count
FROM audit_songs
INNER JOIN audit_albums on audit_songs.album_id = audit_albums.album_id
INNER JOIN honor_single_award as award on award.song_id = audit_songs.song_id
INNER JOIN honor_top_songs as top_songs on audit_songs.song_id::text = top_songs.song_id
where audit_songs.song_id = '31010566'



,0,1
album_id,3109316,3109316
song_id,31010566,31010566
song_name,Sold Out,Sold Out
artist_id,34518,34518
copyright_id,1416336,1416336
deprecated_artist_name,"Jonathan Steingard,Hawk Nelson","Jonathan Steingard,Hawk Nelson"
artist_ids,"35526,34518","35526,34518"
comment_count,126740,126740
artist_array,"[35526, 34518]","[35526, 34518]"
album_name,Diamonds,Diamonds


In [ ]:
start = time.time()
conn_params['database'] = "qqmusicv2"
# sql_query = """SELECT song_mid, song_name, album_mid, album_name, artist_mid, singer_names,
# comment_number, singer_mids, company, release_date
# FROM songs
# WHERE '002Vj8o12l10eK' = ANY(singer_mids)
# or '001fbZH44WRuKW' = ANY(singer_mids)
# or '000Eqw823oivl5' = ANY(singer_mids)

# """
sql_query = """SELECT song_mid, song_name, album_mid, album_name, artist_mid, singer_names,
comment_number, singer_mids, company, release_date
FROM songs
WHERE song_name ilike '%SuperFX%'
"""
# sql_query = "select * from songs limit 2"


with DatabaseConnection(conn_params) as conn:
    df = execute_sql_query(sql_query, conn)
    print(df.shape)
    
end = time.time()
print("time elapsed is: ", end - start)
df.head().T